### Produce classification matrix and report table

In [ ]:
ishp = '/home/jakim/autocls/pureobj.shp'

lulc_col = 'idclcplus'

refs = '/home/jakim/autocls/thresholdsref'

clsrst = '/home/jakim/autocls/thresholdstst'

outtbl = '/home/jakim/autocls/thresholdsref'

In [ ]:
fcol = [
    "threshold",
    'pos_TP', 'pos_FP', 'pos_FN', 'pos_TN', 
    'pos_error', 'pos_acc', 'pos_tpr', 'pos_tnr', 'pos_prec',
    'pos_fpr', 'pos_mcc', 'pos_f05', 'pos_f1', 'pos_f2',
    'neg_TP', 'neg_FP', 'neg_FN', 'neg_TN',
    'neg_error', 'neg_acc', 'neg_tpr', 'neg_tnr', 'neg_prec',
    'neg_fpr', 'neg_mcc', 'neg_f05', 'neg_f1', 'neg_f2'
]

In [ ]:
import pandas as pd
import os

from glass.rd.shp import shp_to_obj
from glass.pys.oss import lst_folders_subfiles, lst_ff, fprop
from glass.rst.confmtx import bin_confmxt_fmrst
from glass.wt import obj_to_tbl

In [ ]:
shpdf = shp_to_obj(ishp)

# Get classes
lulcs = list(shpdf[lulc_col].unique())

In [ ]:
refs_files = lst_ff(refs, file_format='.tif')

ref_rst = {cls : {} for cls in lulcs}

for f in refs_files:
    _, _cls, ispos = fprop(f, 'fn').split('_')
    
    ref_rst[int(_cls)][ispos] = f

In [ ]:
clsfiles = lst_folders_subfiles(clsrst, files_format='.tif')

clspos, clsneg = {}, {}
for k in clsfiles:
    clspos[k], clsneg[k] = {}, {}

    for f in clsfiles[k]:
        fparts = fprop(f, 'fn').split('_')
        ispos, _th = fparts[-2], fparts[-1]
        
        th = float(f"{_th[0]}.{_th[1]}") if len(_th) == 2 else \
            float(f"-{_th[1]}.{_th[2]}")

        if ispos == 'pos':
            clspos[k][th] = f
        else:
            clsneg[k][th] = f

In [ ]:
ref_rst

In [ ]:
# Create confusion matrices

# For each class, create table with all
# thresholds values
for cls in ref_rst:
    cls_sheets = []
    sheets_names = []
    for f in clsfiles:
        ftbl = []
        for th in clspos[f]:
            mtx_pos, eval_pos = bin_confmxt_fmrst(ref_rst[cls]['pos'], clspos[f][th])
            mtx_neg, eval_neg = bin_confmxt_fmrst(ref_rst[cls]['neg'], clsneg[f][th])

            row = [th]

            for i, r in mtx_pos.iterrows():
                row.append(r.positives)
                row.append(r.negatives)

            for i, r in eval_pos.iterrows():
                row.append(r['value'])

            for i, r in mtx_neg.iterrows():
                row.append(r.positives)
                row.append(r.negatives)

            for i, r in eval_neg.iterrows():
                row.append(r['value'])

            ftbl.append(row)

        cls_sheets.append(pd.DataFrame(ftbl, columns=fcol))
        sheets_names.append(f)

    # Write XLSX file
    obj_to_tbl(
        cls_sheets,
        os.path.join(outtbl, f'cls_{str(cls)}.xlsx'),
        sheetsName=sheets_names
    )